# Prediction of Ransome Activity 

Rihem Mansri, Mohamed Issa, Mootez Dakhlaoui, Bourhan Dernayka, Joel Pascal Soffo, Ronny Tonato

In [1]:
import os
import warnings
import pickle
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
import pdb
import datetime
from scipy.stats import pearsonr

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, log_loss, plot_roc_curve, f1_score, precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
import matplotlib
from prettytable import PrettyTable
from pprint import pprint

warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/Data Camp/Data Camp group project

/content/drive/MyDrive/Data Camp/Data Camp group project


In [12]:
# !python download_data.py

## **Importing Data**

TO DO LIST : 
<br>
-Treat the address column
<br>
-Treat year and day columns
<br>
-normalize the data

In [29]:
data = pd.read_csv("BitcoinHeistData.csv")

In [30]:
data.head(n=15)

,address,year,day,length,weight,count,looped,neighbors,income,label
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,8.333333e-03,1,0,2,100050000.0,princetonCerber
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,2.441406e-04,1,0,1,100000000.0,princetonLocky
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000e+00,1,0,2,200000000.0,princetonCerber
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,3.906250e-03,1,0,2,71200000.0,princetonCerber
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,7.284841e-02,456,0,1,200000000.0,princetonLocky
5,112AmFATxzhuSpvtz1hfpa3Zrw3BG276pc,2016,96,144,8.461400e-02,2821,0,1,50000000.0,princetonLocky
6,112E91jxS2qrQY1z78LPWUWrLVFGqbYPQ1,2016,225,142,2.088519e-03,881,0,2,100000000.0,princetonCerber
7,112eFykaD53KEkKeYW9KW8eWebZYSbt2f5,2016,324,78,3.906250e-03,1,0,2,100990000.0,princetonCerber
8,112FTiRdJjMrNgEtd4fvdoq3TC33Ah5Dep,2016,298,144,2.302828e+00,4220,0,2,80000000.0,princetonCerber
9,112GocBgFSnaote6krx828qaockFraD8mp,2016,62,112,3.725290e-09,1,0,1,50000000.0,princetonLocky


In [18]:
# Global view for the data
data.describe()

,year,day,length,weight,count,looped,neighbors,income
count,2.916697e+06,2.916697e+06,2.916697e+06,2.916697e+06,2.916697e+06,2.916697e+06,2.916697e+06,2.916697e+06
mean,2.014475e+03,1.814572e+02,4.500859e+01,5.455192e-01,7.216446e+02,2.385067e+02,2.206516e+00,4.464889e+09
std,2.257398e+00,1.040118e+02,5.898236e+01,3.674255e+00,1.689676e+03,9.663217e+02,1.791877e+01,1.626860e+11
min,2.011000e+03,1.000000e+00,0.000000e+00,3.606469e-94,1.000000e+00,0.000000e+00,1.000000e+00,3.000000e+07
25%,2.013000e+03,9.200000e+01,2.000000e+00,2.148438e-02,1.000000e+00,0.000000e+00,1.000000e+00,7.428559e+07
50%,2.014000e+03,1.810000e+02,8.000000e+00,2.500000e-01,1.000000e+00,0.000000e+00,2.000000e+00,1.999985e+08
75%,2.016000e+03,2.710000e+02,1.080000e+02,8.819482e-01,5.600000e+01,0.000000e+00,2.000000e+00,9.940000e+08
max,2.018000e+03,3.650000e+02,1.440000e+02,1.943749e+03,1.449700e+04,1.449600e+04,1.292000e+04,4.996440e+13


## **Feature Engineering**

Checking missing values

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2916697 entries, 0 to 2916696
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   address    object 
 1   year       int64  
 2   day        int64  
 3   length     int64  
 4   weight     float64
 5   count      int64  
 6   looped     int64  
 7   neighbors  int64  
 8   income     float64
 9   label      object 
dtypes: float64(2), int64(6), object(2)
memory usage: 222.5+ MB


In [22]:
data.isnull().sum()

address      0
year         0
day          0
length       0
weight       0
count        0
looped       0
neighbors    0
income       0
label        0
dtype: int64

In [24]:
data[data.isnull().any(axis="columns")].style.highlight_null()

,address,year,day,length,weight,count,looped,neighbors,income,label


In [26]:
income = data['income'].copy()
np.isnan(income).sum()

0

Treating columns day and year

We create a function that given the year and the day in that year, outputs the day/month/year corresponding
<br>
For example, (year:2016, day:132) => (12/05/2016))

In [34]:
import datetime

year = 1996
days = 32

date = datetime.date(year, 1, 1) #Will give 1996-01-01
delta = datetime.timedelta(days - 1) #str(delta) will be '31 days, 0:00:00'
newdate = date + delta

str(newdate)

'1996-02-01'

In [57]:
years = np.array(data['year'].copy())
days = np.array(data['day'].copy())
years_and_days = np.zeros((len(years), 2))
years_and_days[:,0] = years
years_and_days[:,1] = days

In [61]:
# dates = []
# for year, day in years_and_days:
#   year = int(year)
#   days = int(day)
#   date = datetime.date(year, 1, 1) 
#   delta = datetime.timedelta(days - 1) #str(delta) will be '31 days, 0:00:00'
#   newdate = date + delta
#   dates = dates + [newdate]

In [74]:
# sub_data = data[['year', 'day']]

In [75]:
# sub_data_modif = sub_data.drop_duplicates()

In [76]:
# sub_data_modif

In [79]:
from datetime import date, datetime

In [80]:
dat_temp = []
for i in range(data.shape[0]):
    aux = date.fromordinal(date(data['year'][i], 1, 1).toordinal() + data['day'][i]  - 1).strftime('%d/%m/%Y')
    dat_temp.append(aux)
data['time'] = dat_temp
data['time'] = pd.to_datetime(data['time'])

In [83]:
data.pop('day')
data.pop('year')

0          2017
1          2016
2          2016
3          2016
4          2016
           ... 
2916692    2018
2916693    2018
2916694    2018
2916695    2018
2916696    2018
Name: year, Length: 2916697, dtype: int64

In [85]:
data.pop('address')

0           111K8kZAEnJg245r2cM6y9zgJGHZtJPy6
1          1123pJv8jzeFQaCV4w644pzQJzVWay2zcA
2          112536im7hy6wtKbpH1qYDWtTyMRAcA2p7
3          1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7
4          1129TSjKtx65E35GiUo4AYVeyo48twbrGX
                          ...                
2916692    12D3trgho1vJ4mGtWBRPyHdMJK96TRYSry
2916693    1P7PputTcVkhXBmXBvSD9MJ3UYPsiou1u2
2916694    1KYiKJEfdJtap9QX2v9BXJMpz2SfU4pgZw
2916695    15iPUJsRNZQZHmZZVwmQ63srsmughCXV4a
2916696    3LFFBxp15h9KSFtaw55np8eP5fv6kdK17e
Name: address, Length: 2916697, dtype: object

In [88]:
y = data.pop('label')

In [89]:
cnt_featnames = ['length', 'weight', 'count', 'looped', 'neighbors', 'income']
date_featnames = ['time']

In [90]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pandas import Timestamp

# First we create a scikit-learn encoder that computes 
#  the age in days of columns containing dates
class AgeEncoder(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        self.today = Timestamp.today()
        return self

    def transform(self, X):
        return X.apply(lambda x: (x - self.today).dt.days, axis=0)

# Centers and reduces (variance=1) columns
standard_scaler = StandardScaler()

# One-hot encode, similar to pd.get_dummies
# one_hot_encoder = OneHotEncoder(drop='first', sparse=False)

# A pipeline that first computes age, and standardizes it
scaled_age_encoder = Pipeline([
    ('age', AgeEncoder()),
    ('scaling', StandardScaler())
])

# Let's combine all these transformations
transformer = ColumnTransformer([
    ('standard_scaling', standard_scaler, cnt_featnames),
    ('dates_age_scaled', scaled_age_encoder, date_featnames)
])

data_prep = transformer.fit_transform(data)